In [1]:
# Some tests of NuSTAR lightcurves with hk data
# 
# 26-04-2023    IGH

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy import timeseries as ts
from sunpy.time import parse_time
from astropy.io import fits
import astropy.units as u
import datetime
import pandas as pd
import os
from pathlib import Path
import glob
from stixdcpy.quicklook import LightCurves
from stixdcpy.net import Request as jreq
from stixdcpy import auxiliary as aux

import nsx_func

import warnings
warnings.simplefilter('ignore')

plt.rcParams.update({'font.size': 18,'font.family':"sans-serif",\
            'font.sans-serif':"Arial",'mathtext.default':"regular"})

In [3]:
# List of all the NuSTAR observations, with my naming, 
# based off first day of observing per campaign
dobs=['20140910','20141101','20141211',
    '20150429','20150901',
    '20160219','20160422','20160726',
    '20170321','20170821','20170911','20171010',
    '20180529','20180907','20180928',
    '20190112','20190412','20190425','20190702',
    '20200129','20200221','20200606','20200912',
    '20210108','20210429','20210720','20210730','20211117',
    '20220224','20220603','20220906','20221209',
    '20230318']

# Data directory
# ddir=str(Path.home())+'/data/heasarc_nustar/'
ddir='/Volumes/Samsung_T5/data/heasarc_nustar/'

In [25]:
for icamp in [21,23,24,25,26,27,28,29,30,31,32]:
# for icamp in [23]:
    maindir=ddir+'ns_'+dobs[icamp]+'/'
    print(icamp, ' --- ',maindir)
    # Most start with 20 or 80 or 90?
    ids = [f.name for f in os.scandir(maindir) \
        if (f.is_dir() and (f.name.startswith('20') or f.name.startswith('80') or f.name.startswith('90')))]
    ids=sorted(ids)
    # Filter out random other dirs
    ids=[f for f in ids if not f.endswith('_new')]
    # Remove the mosiac from 20220603
    if (dobs[icamp] == '20220603'):
        ids=[f for f in ids if f.startswith('2062400')]      
    print("#: ",len(ids),' --- ',ids)

    # # STIX + GOES/XRS + NuSTAR  lightcurves
    for nsid in ids:
        print(nsid)

        df_test=nsx_func.nsrate(maindir=maindir,nsid=nsid,hk=True)#,englow=2,enghigh=10)
        df10 = df_test.resample('10s', level=0).mean()
        # yrange for NuSTAR plot
        alldata=np.concatenate((df10["rta"].values,df10["rtb"].values))
        alldata=alldata[alldata > 0]
        maxy=1.4*max(alldata)
        miny=0.98*min(alldata)
        yr=[miny,maxy]
        # alllvt=np.concatenate((df10["lvta"].values,df10["lvtb"].values))
        # alllvt=alllvt[alllvt > 0]
        # lvtyr=[0.9*min(alllvt),1.2*max(alllvt)]
        # allacc=np.concatenate((df10["acca"].values,df10["accb"].values))
        # allacc=allacc[allacc > 0]
        # accyr=[0.98*min(allacc),1.4*max(allacc)]
        # allrej=np.concatenate((df10["reja"].values,df10["rejb"].values))
        # allrej=allrej[allrej > 0]
        # rejyr=[0.98*min(allrej),1.4*max(allrej)]

        # Time range to nearest 5mins
        mint=df10.index[0].to_pydatetime()
        mint-=datetime.timedelta(minutes=5,seconds=mint.second)
        mint-=datetime.timedelta(minutes=mint.minute % 5)
        maxt=df10.index[-1].to_pydatetime()
        maxt+=datetime.timedelta(minutes=5,seconds=60-maxt.second)
        maxt-=datetime.timedelta(minutes=maxt.minute % 5)
        tr=[mint,maxt]
 

        # Control the time axis labelling
        myFmt = matplotlib.dates.DateFormatter('%H:%M')
        majorx= matplotlib.dates.MinuteLocator(interval=10)
        minorx= matplotlib.dates.MinuteLocator(interval=1)
        plt.rcParams.update({'font.size': 14})
        fig, axs= plt.subplots(4,figsize=(8, 11))

        axs[0].plot(df10.index,df10["lvta"].values,drawstyle='steps-post',\
            lw=2,color='rebeccapurple',label='A')
        axs[0].plot(df10.index,df10["lvtb"].values,drawstyle='steps-post',\
            lw=2,color='teal',label='B')
        # mnlvt=np.mean(lvtyr)
        # lvtpow = int(f"{mnlvt:e}".split('e')[1])
        # mnlvt=round(mnlvt/10**lvtpow)*10**lvtpow
        # if (mnlvt < lvtyr[0]):   
        #     lvtyr[0] = mnlvt
        # if (mnlvt > lvtyr[1]):   
        #     lvtyr[1] = mnlvt
        # axs[0].set_ylim(lvtyr)
        axs[0].set_ylim([5e-4,1])
        axs[0].set_yscale('log')
        # axs[0].plot(tr,[mnlvt,mnlvt],linestyle='dotted',color='grey')
        # axs[0].annotate(f'  {mnlvt*100:.2f}%',(tr[0],1.05*mnlvt),
                        # color='grey',fontweight="bold",rotation='horizontal')
        axs[0].set_ylabel('Livetime')
        # axs[1].ticklabel_format(axis='y', style='sci', scilimits=(0,0), useOffset=True)


        axs[1].plot(df10.index,df10["acca"].values,drawstyle='steps-post',\
            lw=2,color='rebeccapurple',label='A')
        axs[1].plot(df10.index,df10["accb"].values,drawstyle='steps-post',\
            lw=2,color='teal',label='B')
        axs[1].set_ylim([1.5,500])
        # axs[1].set_ylim(accyr)
        axs[1].set_yscale('log')
        axs[1].set_ylabel('Accept [count$\;s^{-1}$]')

        axs[2].plot(df10.index,df10["reja"].values,drawstyle='steps-post',\
            lw=2,color='rebeccapurple',label='A')
        axs[2].plot(df10.index,df10["rejb"].values,drawstyle='steps-post',\
            lw=2,color='teal',label='B')
        axs[2].set_ylim([1.5,7e3])
        axs[2].set_yscale('log')
        # axs[2].set_ylim(rejyr)
        axs[2].set_ylabel('Reject [count$\;s^{-1}$]')

        axs[3].plot(df10.index,df10["rta"].values,drawstyle='steps-post',\
            lw=2,color='rebeccapurple',label='A')
        axs[3].plot(df10.index,df10["rtb"].values,drawstyle='steps-post',\
            lw=2,color='teal',label='B')
        axs[3].set_xlabel(f'Start Time {mint:%d-%b-%Y %H:%M:%S}')
        axs[3].set_ylabel('NuSTAR [count$\;s^{-1}$]')
        axs[3].set_ylim(yr)
        axs[3].legend(loc=2,fontsize=14,ncols=2)
        axs[3].ticklabel_format(axis='y', style='sci', scilimits=(0,0), useOffset=True)

        for ax in axs:
            ax.set_xlim(tr)
            ax.xaxis.set_major_locator(majorx)
            ax.xaxis.set_minor_locator(minorx)
            ax.xaxis.set_major_formatter(myFmt)
            ax.grid(visible=True,color='silver',axis='y',which='both',linestyle='solid',alpha=0.5)
            ax.grid(visible=True,color='silver',axis='x',which='major',linestyle='solid',alpha=0.5)

        # axs[0].grid(visible=True,axis='y',which='minor',color='grey',linestyle=':',alpha=0.5)
        plt.annotate(nsid,(0.01,0.01),xycoords='figure fraction',fontsize=14)

        fig.subplots_adjust(bottom=0.06, top=0.97, left=0.12, right=0.95)
        plt.savefig('figs/ns_'+dobs[icamp]+f'/hkltc_{mint:%Y%m%d_%H%M}_'+nsid+'_ngs.png')
        plt.close()
        # break
    # break


21  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20200606/
#:  10  ---  ['20611001001', '20611002001', '20611003001', '20611004001', '20611005001', '20611006001', '20611007001', '20611008001', '20611009001', '20611010001']
20611001001
20611002001
20611003001
20611004001
20611005001
20611006001
20611007001
20611008001
20611009001
20611010001
23  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20210108/
#:  12  ---  ['20612001001', '20612002001', '20612003001', '20612004001', '20613001001', '20613002001', '20613003001', '20613004001', '20613005001', '20614001001', '20614002001', '20614003001']
20612001001
20612002001
20612003001
20612004001
20613001001
20613002001
20613003001
20613004001
20613005001
20614001001
20614002001
20614003001
24  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20210429/
#:  16  ---  ['20615001001', '20615002001', '20615003001', '20615004001', '20615005001', '20616001001', '20616002001', '20616003001', '20616004001', '20616005001', '20617001001', '20617002001'